In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import manifold
from sklearn.manifold import TSNE
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import NullFormatter
from sklearn.decomposition import PCA

from time import time

In [36]:
df = pd.read_csv('batchcorrected_log2cpm.tsv', sep='\t',index_col=0)
#df.set_index(df["Unnamed: 0"], inplace=True)
df.head()

,100_2,101_3,102_2,103_3,104_2,105_2,106_4,107_4,109_1,11_4,...,90_2,91_2,92_3,93_2,94_4,95_4,96_3,97_2,98_3,99_1
ENSG00000000003,4.224093,4.088110,4.108960,4.338494,4.144095,3.961678,4.068081,4.181222,4.177308,4.345819,...,4.095915,4.185599,3.984862,4.437899,4.258712,3.980472,4.310913,4.078538,4.171295,4.225876
ENSG00000000419,4.273573,4.278318,4.508365,4.533990,4.451600,4.437512,4.229555,4.359195,4.299986,4.569697,...,4.615619,4.413496,4.623100,4.781393,4.534774,4.570218,4.722600,4.492435,4.560072,4.415542
ENSG00000000457,4.716466,4.969749,4.852762,4.947713,4.882771,4.704028,4.675137,4.517319,4.740297,4.672500,...,4.700506,4.839435,4.763086,4.870351,4.876640,4.812464,4.812821,4.840323,4.880159,4.652706
ENSG00000000460,4.126170,4.528228,4.286024,4.310150,4.150902,4.265058,4.737984,4.102466,4.593532,4.613472,...,4.476497,4.568329,4.556887,4.323269,4.165587,4.667403,4.378113,4.334682,4.361387,3.685774
ENSG00000000938,1.171196,0.862429,1.540143,1.105556,1.375205,1.568452,1.638125,1.233508,1.497463,1.548793,...,1.454494,0.874062,1.199031,2.341509,1.732493,1.523060,1.136348,1.578141,0.897523,1.096682


In [33]:
#transpose the dataframe --> make genes as dementions 
dfn = df.T
dfn['mgs_level'] = 0
dfn.reset_index(inplace = True)
list = dfn['index'].apply(lambda x:x[-1]).tolist()
#list 
dfn['mgs_level'] = list
level = dfn.pop('mgs_level')
dfn.insert(1,'mgs_level',level)
dfn.head()

,index,mgs_level,ENSG00000000003,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,...,ENSG00000283529,ENSG00000283536,ENSG00000283590,ENSG00000283617,ENSG00000283619,ENSG00000283623,ENSG00000283633,ENSG00000283662,ENSG00000283667,ENSG00000283674
0,100_2,2,4.224093,4.273573,4.716466,4.126170,1.171196,4.015480,3.293096,5.294673,...,1.145577,3.068081,1.881444,1.346983,1.513299,0.797770,0.419013,0.843287,1.047956,1.928373
1,101_3,3,4.088110,4.278318,4.969749,4.528228,0.862429,3.843299,3.390655,5.527211,...,1.189183,2.866956,1.795239,2.366877,1.177736,1.070189,1.483662,0.637100,1.125788,1.522671
2,102_2,2,4.108960,4.508365,4.852762,4.286024,1.540143,3.846015,3.596859,4.948842,...,0.915614,2.886454,1.835291,2.017325,1.232213,0.841682,0.010696,0.723624,1.194329,1.378135
3,103_3,3,4.338494,4.533990,4.947713,4.310150,1.105556,3.877232,3.448069,5.472893,...,0.850755,2.891162,1.961135,1.967287,1.947879,0.387205,0.712213,0.256273,1.322104,2.025008
4,104_2,2,4.144095,4.451600,4.882771,4.150902,1.375205,4.456291,3.501590,5.159368,...,0.647376,3.128063,2.011594,1.556070,0.814551,1.100907,1.155770,0.919095,1.381362,1.788005


In [53]:
## separate control vs patient
control = dfn[dfn.mgs_level == '1']
patient = dfn[dfn.mgs_level != '1']

In [66]:
control.head()

,index,mgs_level,ENSG00000000003,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,...,ENSG00000283536,ENSG00000283590,ENSG00000283617,ENSG00000283619,ENSG00000283623,ENSG00000283633,ENSG00000283662,ENSG00000283667,ENSG00000283674,binary
8,109_1,1,4.177308,4.299986,4.740297,4.593532,1.497463,4.057762,3.343734,5.021024,...,2.887815,1.755579,1.772080,0.592089,0.999594,0.705493,0.748250,1.282221,2.053524,0
15,115_1,1,4.241472,4.441022,4.804861,3.989499,1.072882,3.614325,3.297278,5.242712,...,2.686574,1.109814,2.035455,1.718607,1.033493,0.799001,0.487644,1.526137,1.553504,0
17,117_1,1,3.863648,4.437281,4.882508,4.504008,1.671384,4.387065,3.378628,5.138772,...,2.669231,1.659946,1.271946,0.823566,1.191215,0.812745,0.815359,1.579369,2.274689,0
18,119_1,1,4.063576,4.290273,4.466665,4.761237,1.580267,3.809128,3.524963,5.093789,...,2.741063,1.801195,1.801970,2.053347,0.314001,0.743814,0.815210,0.982556,1.984814,0
19,120_1,1,4.049423,4.443427,4.753416,4.395347,1.644460,4.031802,3.125032,5.148547,...,3.036836,2.161906,1.865505,1.353617,1.873662,0.524925,0.626578,0.596844,1.321599,0


In [68]:
patient.head()

,index,mgs_level,ENSG00000000003,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,...,ENSG00000283536,ENSG00000283590,ENSG00000283617,ENSG00000283619,ENSG00000283623,ENSG00000283633,ENSG00000283662,ENSG00000283667,ENSG00000283674,binary
0,100_2,2,4.224093,4.273573,4.716466,4.126170,1.171196,4.015480,3.293096,5.294673,...,3.068081,1.881444,1.346983,1.513299,0.797770,0.419013,0.843287,1.047956,1.928373,1
1,101_3,3,4.088110,4.278318,4.969749,4.528228,0.862429,3.843299,3.390655,5.527211,...,2.866956,1.795239,2.366877,1.177736,1.070189,1.483662,0.637100,1.125788,1.522671,1
2,102_2,2,4.108960,4.508365,4.852762,4.286024,1.540143,3.846015,3.596859,4.948842,...,2.886454,1.835291,2.017325,1.232213,0.841682,0.010696,0.723624,1.194329,1.378135,1
3,103_3,3,4.338494,4.533990,4.947713,4.310150,1.105556,3.877232,3.448069,5.472893,...,2.891162,1.961135,1.967287,1.947879,0.387205,0.712213,0.256273,1.322104,2.025008,1
4,104_2,2,4.144095,4.451600,4.882771,4.150902,1.375205,4.456291,3.501590,5.159368,...,3.128063,2.011594,1.556070,0.814551,1.100907,1.155770,0.919095,1.381362,1.788005,1


In [67]:
## 0 if stage is 1, 1 if stage is 1,2,3
control["binary"] = 0
patient["binary"] = 1

/Users/minjunp/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/minjunp/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [76]:
## merge two datasets vertically and remove mgs level
new_df = control.append(patient, ignore_index=True)
new_df = new_df.iloc[:,2:]
new_df.head()

,ENSG00000000003,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,ENSG00000001460,...,ENSG00000283536,ENSG00000283590,ENSG00000283617,ENSG00000283619,ENSG00000283623,ENSG00000283633,ENSG00000283662,ENSG00000283667,ENSG00000283674,binary
0,4.177308,4.299986,4.740297,4.593532,1.497463,4.057762,3.343734,5.021024,5.864690,5.190097,...,2.887815,1.755579,1.772080,0.592089,0.999594,0.705493,0.748250,1.282221,2.053524,0
1,4.241472,4.441022,4.804861,3.989499,1.072882,3.614325,3.297278,5.242712,5.836880,5.064816,...,2.686574,1.109814,2.035455,1.718607,1.033493,0.799001,0.487644,1.526137,1.553504,0
2,3.863648,4.437281,4.882508,4.504008,1.671384,4.387065,3.378628,5.138772,5.641097,5.117710,...,2.669231,1.659946,1.271946,0.823566,1.191215,0.812745,0.815359,1.579369,2.274689,0
3,4.063576,4.290273,4.466665,4.761237,1.580267,3.809128,3.524963,5.093789,5.955642,5.215154,...,2.741063,1.801195,1.801970,2.053347,0.314001,0.743814,0.815210,0.982556,1.984814,0
4,4.049423,4.443427,4.753416,4.395347,1.644460,4.031802,3.125032,5.148547,5.687160,5.067885,...,3.036836,2.161906,1.865505,1.353617,1.873662,0.524925,0.626578,0.596844,1.321599,0


In [78]:
## print shape
new_df.shape

(453, 18054)

In [88]:
## split into X, Y variables
X_train = new_df.iloc[:,:-1]
y_train = new_df.iloc[:,-1]
X_train.shape, Y_train.shape

((453, 18053), (453,))

In [86]:
import seaborn as sns
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [89]:
sel_ = SelectFromModel(LogisticRegression(C=1, penalty='l1'))
sel_.fit(scaler.transform(X_train), y_train)

/Users/minjunp/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='warn',
                                             n_jobs=None, penalty='l1',
                                             random_state=None, solver='warn',
                                             tol=0.0001, verbose=0,
                                             warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [90]:
sel_.get_support()

array([False, False, False, ...,  True, False, False])

In [91]:
## Make a list of with the selected features.
selected_feat = X_train.columns[(sel_.get_support())]
print('total features: {}'.format((X_train.shape[1])))
print('selected features: {}'.format(len(selected_feat)))
print('features with coefficients shrank to zero: {}'.format(
      np.sum(sel_.estimator_.coef_ == 0)))

total features: 18053
selected features: 305
features with coefficients shrank to zero: 17748


In [92]:
## Number of features which coefficient was shrank to zero :
np.sum(sel_.estimator_.coef_ == 0)

17748

In [93]:
## Identifying the removed features
removed_feats = X_train.columns[(sel_.estimator_.coef_ == 0).ravel().tolist()]
removed_feats

Index(['ENSG00000000003', 'ENSG00000000419', 'ENSG00000000457',
       'ENSG00000000460', 'ENSG00000000938', 'ENSG00000000971',
       'ENSG00000001036', 'ENSG00000001084', 'ENSG00000001167',
       'ENSG00000001460',
       ...
       'ENSG00000283515', 'ENSG00000283529', 'ENSG00000283536',
       'ENSG00000283590', 'ENSG00000283617', 'ENSG00000283619',
       'ENSG00000283623', 'ENSG00000283633', 'ENSG00000283667',
       'ENSG00000283674'],
      dtype='object', length=17748)

In [95]:
## Removing the features from training
X_train_selected = sel_.transform(X_train)
X_train_selected.shape

(453, 305)